# Azure AI Agent Service with Bing Grounding

이 노트북은 Azure AI Agent Service와 Bing Grounding Tool을 활용하여, 사용자 질문에 대해 Bing 검색 결과를 기반으로 답변을 생성하는 예제를 다룹니다.

## 주요 내용

- Azure AI Project Client와 Bing Grounding Tool을 초기화합니다.
- Bing 연결 정보를 불러와서 Grounding Tool을 구성합니다.
- 사용자 프롬프트(질문)를 입력하면, Bing 검색을 활용한 답변을 생성하는 Agent를 동적으로 생성하고 실행합니다.
- 각 질문에 대해 에이전트의 답변과 참고한 URL 정보를 출력합니다.

이 과정을 통해, 실제 Bing 검색 결과를 기반으로 한 최신 정보 답변을 Azure AI Agent를 통해 자동화하는 방법을 실습할 수 있습니다.





In [48]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageRole, BingGroundingTool
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import AzureOpenAI
import json
load_dotenv(override=True, dotenv_path="../.env")

credential = DefaultAzureCredential()


In [43]:
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ["AZURE_AI_PROJECT_CONNECTION_STRING"],
)

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"],
    include_credentials=True
)
conn_id = bing_connection.id

print(conn_id)

bing = BingGroundingTool(connection_id=conn_id)

def populate_answer_agent_with_bing_grounding_tool(prompt: str, project_client: AIProjectClient, bing):
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘.",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
        top_p=0.9,
    )

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=prompt,
        
    )

    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, max_completion_tokens=1500)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    project_client.agents.delete_agent(agent.id)
    response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
        MessageRole.AGENT
    )

    return  response_message



def display_markdown_content(content: str):
    display(Markdown(content))

/subscriptions/ef750ece-3939-4a4b-ade4-cae9eabcf312/resourceGroups/rg-writer/providers/Microsoft.MachineLearningServices/workspaces/ai-project-q4u67epqs2xo4/connections/agentbingsearch


In [ ]:
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.
            
            1. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘            
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]



for input in inputs:
    print(f"User input: {input}")
    rewritten_query = rewrite_query_for_search_and_llm(input, client)
    response_message = populate_answer_agent_with_bing_grounding_tool(rewritten_query['llm_query'], project_client, bing)
    if response_message:
        for text_message in response_message.text_messages:
            display_markdown_content(text_message.text.value)
        # for annotation in response_message.url_citation_annotations:
        #     print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")


User input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘


삼성전자 인덕션 제품 중에서 2구 모델이 아닌 3구 이상 또는 다양한 화구 수를 가진 인덕션 제품을 추천해 드리겠습니다. 삼성전자 인덕션은 화구 수와 기능에 따라 다양하게 출시되어 주방 환경과 요리 스타일에 맞게 선택할 수 있습니다.

### 1. 삼성전자 인덕션 3구 모델 (예: NZ63R3707BK)
- **주요 기능**
  - 3개의 화구로 동시 조리 가능, 대가족이나 다양한 요리 시 편리
  - 인공지능 센서 탑재로 최적화된 화력 조절
  - 프리존 기능: 넓은 공간을 자유롭게 활용 가능
  - 잔열 표시 및 자동 안전 차단 기능 내장
- **장점**
  - 다양한 크기 용기 사용 가능
  - 높은 효율의 열 전달로 빠른 조리 시간
  - 안전성과 편의성을 모두 갖춤

### 2. 삼성전자 인덕션 4구 모델 (예: NZ64R5707FK)
- **주요 기능**
  - 4개의 독립 화구로 대용량 요리 가능
  - 스마트 터치 컨트롤로 직관적 조작
  - 고화력 버너 포함, 빠른 가열 가능
  - 타이머, 예약 기능 내장
- **장점**
  - 대가족 및 요리량이 많은 사용자에게 적합
  - 다양한 요리 동시 진행으로 시간 절약
  - 디자인이 깔끔하고 세련되어 주방 인테리어에 어울림

### 3. 삼성전자 인덕션 5구 이상 모델 (예: NZ75R8707GK)
- **주요 기능**
  - 5구 이상의 대형 인덕션으로 프로페셔널한 요리 환경 제공
  - 멀티존 지원으로 다양한 크기와 형태의 조리기구 호환
  - 스마트 연동 기능으로 IoT 제어 가능 (스마트홈 연동)
  - 고급 안전 기능과 자동 청소 모드
- **장점**
  - 전문 요리사나 요리 애호가에게 이상적
  - 다양한 조리 옵션과 편리한 관리 기능
  - 주방 전체를 하나의 요리 스테이션으로 활용 가능

---

이 외에도 삼성전자는 다양한 크기와 기능의 인덕션 모델을 출시하고 있어, 사용자의 주방 크기와 조리 습관에 맞춘 최적의 선택이 가능합니다. 필요하시면 구체적인 모델별 비교와 구매처 안내도 도와드릴 수 있습니다.

User input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘


부모님께 선물하기 좋은 삼성전자 TV를 추천드리겠습니다. 부모님 사용에 적합한 TV는 적당한 크기, 뛰어난 화질, 편리한 기능을 갖춘 제품이 좋습니다. 다음은 추천 모델과 주요 특징입니다.

---

### 1. 삼성전자 QN90B Neo QLED 4K 스마트 TV

- **화질**: Neo QLED 기술로 선명하고 생생한 4K 화질 제공. 미니 LED 백라이트로 깊은 명암비와 풍부한 색감이 특징.
- **크기**: 43~65인치 등 다양한 크기 옵션으로 공간에 맞게 선택 가능. 부모님 방이나 거실에 맞는 적당한 크기 선택 가능.
- **기능**:
  - 스마트 TV 기능으로 유튜브, 넷플릭스 등 다양한 OTT 서비스 손쉽게 이용 가능.
  - 음성 인식 리모컨 지원으로 리모컨 버튼 찾기 어려운 분들도 간편 조작.
  - 눈 건강을 고려한 눈부심 방지 기술 탑재.
- **장점**: 뛰어난 화질과 사용자 편의성, 고급스러운 디자인으로 부모님 TV로 적합.

---

### 2. 삼성전자 The Frame 43~55인치

- **화질**: QLED 4K 화질로 선명하고 자연스러운 색감을 제공.
- **크기**: 43~55인치 크기 선택 가능, 방이나 거실에 적합한 크기.
- **기능**:
  - TV를 껐을 때는 명화나 사진 액자처럼 인테리어 효과를 주는 '아트 모드' 탑재.
  - 슬림한 디자인과 벽걸이 설치 가능, 공간 활용에 좋음.
  - 스마트 TV 기능과 음성 제어 지원.
- **장점**: TV 겸 인테리어 효과로 집안 분위기 살리고 싶은 분께 추천.

---

### 3. 삼성전자 AU8000 Crystal UHD TV

- **화질**: Crystal UHD 4K 화질로 선명하고 깨끗한 화면 제공.
- **크기**: 43~65인치 다양한 크기.
- **기능**:
  - 심플하고 직관적인 스마트 TV 인터페이스로 부모님도 쉽게 사용 가능.
  - 음성 리모컨 지원, 주요 OTT 앱 내장.
  - 슬림한 디자인과 넓은 시야각.
- **장점**: 가성비 좋은 4K TV로, 기본기에 충실해 부담 없이 선물하기 좋은 모델.

---

### 요약

| 모델명                 | 크기 옵션       | 주요 특징                         | 추천 이유                       |
|-----------------------|----------------|---------------------------------|------------------------------|
| QN90B Neo QLED        | 43~65인치      | 미니 LED, 뛰어난 명암비, 스마트 기능 | 고급 화질과 편리함 선호 시       |
| The Frame             | 43~55인치      | 아트 모드, 인테리어 효과, QLED 4K | 인테리어와 기능 모두 원하는 분께 |
| AU8000 Crystal UHD    | 43~65인치      | 가성비 좋은 4K, 심플 UI         | 가격 부담 적고 기본기 충실한 TV |

---

부모님께서 TV를 주로 어디에 두시는지, 크기 선호, 특별히 원하시는 기능이 있는지 알려주시면 더 맞춤 추천도 가능합니다. 필요하면 추가 안내 드릴게요!

User input: 삼성전자 25년 제품이 작년 대비 좋아진것은


...

삼성전자가 소유한 오디오 브랜드인 JBL과 하만카돈(Harman Kardon)은 모두 하만 인터내셔널(Harman International)이 소유한 브랜드이며, 하만은 삼성전자가 2017년에 인수한 회사입니다. 두 브랜드는 모두 고품질 오디오 제품을 생산하지만, 제품군, 음향 특성, 기술적 포커스, 그리고 브랜드 이미지 및 타겟 소비자층에 있어 차이가 있습니다.

---

## 1. 브랜드 개요 및 소유

- **JBL**  
  - 하만 인터내셔널 소유의 대중적 오디오 브랜드  
  - 삼성전자 하만 인수 후 함께 포함됨  
  - 주로 다용도, 휴대용 스피커, 프로 오디오 장비까지 다양한 제품군 보유  

- **하만카돈 (Harman Kardon)**  
  - 하만의 고급 오디오 브랜드 중 하나  
  - 프리미엄 홈 오디오, 카 오디오, 사운드바 등에 주력  
  - 세련된 디자인과 고급스러운 사운드 경험 제공에 집중  

---

## 2. 제품군 집중

| 브랜드 | 주요 제품군 | 특징 및 활용도 |
|---------|-------------|----------------|
| JBL     | 휴대용 블루투스 스피커, 헤드폰, 프로용 PA 시스템, 자동차 오디오 | 스포츠, 야외 활동용 내구성 좋은 제품부터 콘서트 등 프로페셔널 장비까지 폭넓음 |
| 하만카돈 | 홈 오디오 시스템, 사운드바, 앰프, 고급 헤드폰, 자동차 오디오 | 고급스러운 인테리어와 조화를 이루는 디자인, 하이파이 음향에 중점 |

---

## 3. 음향 특성 및 기술적 차이점

- **JBL**  
  - 베이스 강화, 강력한 출력, 클리어하고 다이내믹한 사운드  
  - 휴대성과 내구성 강조 (방수, 충격 방지 기능 등)  
  - 다이나믹 드라이버와 JBL의 특허 기술로 강력한 저음 구현  
  - 사운드의 생동감과 파워감에 중점, 실내외 어디서나 활용 가능  

- **하만카돈**  
  - 중고음의 섬세함과 자연스러운 밸런스가 뛰어남  
  - 음향 왜곡 최소화, 고해상도 오디오 재생 지원  
  - 프리미엄 사운드에 맞춰 음의 투명성과 공간감 구현  
  - 디자인과 소재에서 고급스러움 강조, 하이파이 오디오 애호가 대상  

---

## 4. 브랜드 이미지 및 타겟 소비자층

| 브랜드 | 이미지 | 타겟 소비자 |
|---------|---------|-------------|
| JBL     | 젊고 활기찬 이미지, 모험적이고 활동적인 라이프스타일에 어울림 | 야외 활동을 즐기거나 대중적인 휴대용 음향기기를 원하는 소비자, 프로 오디오 시장 사용자 |
| 하만카돈 | 세련되고 프리미엄 이미지, 인테리어와 조화를 이루는 고급스러운 느낌 | 고품질 사운드와 디자인을 중시하는 홈 오디오 애호가, 프리미엄 자동차 오디오 구매자 |

---

### 요약

- JBL은 **다양한 용도와 환경에서 강력하고 생동감 있는 사운드를 제공**하며, 휴대성과 내구성을 중시하는 제품군이 강점입니다.
- 하만카돈은 **세련된 디자인과 음질 밸런스, 자연스러운 음향을 추구하는 프리미엄 브랜드**로서, 고급 홈 오디오와 자동차 오디오에서 특히 두각을 나타냅니다.

필요하시면 특정 제품 추천이나 기능 비교도 도와드릴 수 있습니다.

User input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요


갤럭시 버즈 이어버드 한 쪽을 새로 구매하여 기존 이어버드와 함께 정상적으로 페어링하는 방법은 다음과 같습니다. 단계별로 자세히 설명드릴게요.

---

## 새 이어버드와 기존 이어버드를 양쪽 모두 정상적으로 사용하기 위한 페어링 절차

### 1단계: 기존 이어버드와 스마트폰 연결 해제
- 기존 갤럭시 버즈가 스마트폰과 연결되어 있다면, 먼저 블루투스 연결을 해제하거나 갤럭시 웨어러블(Galaxy Wearable) 앱에서 기존 버즈 연결을 해제하세요.

### 2단계: 양쪽 이어버드를 충전 케이스에 넣기
- 기존 이어버드와 새 이어버드를 모두 충전 케이스에 넣고, 케이스 뚜껑을 닫으세요.
- 충전 케이스에 이어버드가 제대로 장착되어 있는지 확인합니다.

### 3단계: 충전 케이스에서 이어버드 초기화(리셋)
- 충전 케이스 뚜껑을 연 상태에서 이어버드를 꺼내지 않고, 케이스의 버튼을 약 7초 이상 눌러 초기화를 진행하세요.
- 초기화가 완료되면 이어버드 LED가 깜빡이면서 재설정이 완료됩니다.
- (참고: 초기화 방법은 모델에 따라 다를 수 있으니 사용 중인 갤럭시 버즈 모델별 초기화 방법도 확인하세요.)

### 4단계: 이어버드 페어링 모드 진입
- 이어버드를 케이스에서 꺼내면 자동으로 페어링 모드로 진입합니다.
- 스마트폰 블루투스 설정을 열고, ‘Galaxy Buds (모델명)’이 검색되는지 확인하세요.

### 5단계: 스마트폰과 이어버드 연결
- 검색된 ‘Galaxy Buds (모델명)’를 선택해 페어링합니다.
- 갤럭시 웨어러블 앱이 자동 실행되면, 앱 안내에 따라 이어버드 소프트웨어를 업데이트하고 설정을 완료하세요.

### 6단계: 양쪽 이어버드 정상 작동 확인
- 양쪽 이어버드가 모두 연결되어 소리가 양쪽에서 잘 나오는지 확인하세요.
- 만약 한 쪽에서만 소리가 나면 이어버드 초기화 및 충전 케이스 재장착 후 다시 시도하세요.

---

## 추가 팁
- 새 이어버드를 기존 이어버드와 정상적으로 연결하려면 두 이어버드가 같은 충전 케이스에 있어야 합니다.
- 페어링 문제 발생 시 갤럭시 웨어러블 앱에서 ‘이어버드 초기화’ 기능을 활용하면 도움이 됩니다.
- 이어버드별로 다소 차이가 있을 수 있으니, 제품 매뉴얼을 참고하는 것도 좋습니다.

---

필요한 경우, 사용 중인 갤럭시 버즈 모델명을 알려주시면 좀 더 구체적이고 맞춤형 가이드를 드릴 수 있습니다.

User input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나


삼성전자 모니터 S25 모델과 S24 모델의 무게를 비교해드리겠습니다.

- **삼성전자 S25 모델 무게**: 약 3.5kg (스탠드 포함 기준)
- **삼성전자 S24 모델 무게**: 약 3.1kg (스탠드 포함 기준)

두 모델 간의 무게 차이는 약 **0.4kg** 정도입니다. S25 모델이 S24 모델보다 약간 더 무거운 편입니다.

더 정확한 무게는 각각의 상세 제품 사양서에서 확인할 수 있으니, 필요 시 알려주세요!